<a target="_blank" href="../cluster" style="font-size:20px">All Applications (YARN)</a>

# Домашнее задание

Давайте рассмотрим задачу классификации на следующие два класса:
- 1 для 'American_movie_actors'
- 0 для 'American_stage_actors'

На лекциях мы обсуждали, что вместо словаря можно использовать хэширование.

Вам предлагается проверить, как поведет себя модель после использования хэширования и ответить на следующие вопросы:
1. **Какой roc_auc_score на тестовой выборке получается при использовании словаря?**
2. **Какой roc_auc_score на тестовой выборке получается при переходе со словаря на хэширование?**

Детали:
1. Разбейте выборки на обучающую и тестовую по четности `id` статьи: четные в обучение, нечетные в тест. Только по тренировочной части мы считаем градиенты!
2. Для подсчета roc_auc_score вам нужно получить предсказания и истинные ответы для примеров из тестовой выборки. Все пары (предсказание, ответ) помещаются в память, воспользуйтесь этим!
3. В качестве хэш-функции используйте `murmurhash3_32(x) % 2**20`.
4. Зафиксируйте random seed в начальном приближении весов: `np.random.seed(0); weights = np.random.random(...)`
5. Обучите 500 эпох с шагом 0.3. После каждой эпохи вызывайте `weights_broadcast.destroy()` для удаления broadcast переменной, чтобы не закончилась память. 
6. Вот так выглядит roc_auc_score на тестовой выборке от числа эпох (чем больше roc_auc_score, тем лучше):
<img src="images/test_auc.png" width="600px"></img>

Решение сохраните в файл `result.json`. Пример содержимого файла:

```json
{
    "q1": 0.123,
    "q2": 0.456
}```

In [1]:
from sklearn.utils import murmurhash3_32

In [2]:
from sklearn.metrics import roc_auc_score

# y_true - настоящие классы
# y_score - вероятности класса 1
# https://ru.wikipedia.org/wiki/ROC-кривая
roc_auc_score(y_true=[1, 1, 0, 0], y_score=[0.8, 0.7, 0.3, 0.2])

1.0

In [3]:
# ТУТ ВАШЕ РЕШЕНИЕ

## Решение через мешок слов

In [4]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

### Копируем файлы в HDFS

In [5]:
! hadoop fs -copyFromLocal wiki /

copyFromLocal: `/wiki/wiki.jsonl': File exists
copyFromLocal: `/wiki/README.txt': File exists
copyFromLocal: `/wiki/categories.jsonl': File exists


In [6]:
! hadoop fs -ls -h /wiki

Found 3 items
-rw-r--r--   1 jovyan supergroup        387 2021-08-23 10:58 /wiki/README.txt
-rw-r--r--   1 jovyan supergroup     60.9 M 2021-08-23 10:58 /wiki/categories.jsonl
-rw-r--r--   1 jovyan supergroup    143.4 M 2021-08-23 10:58 /wiki/wiki.jsonl


### Подготавливаем таблицу для обучения

In [7]:
import numpy as np
from collections import Counter

In [42]:
# Загружаем обе таблицы в dataframe
wiki = se.read.json("hdfs:///wiki/wiki.jsonl")
wiki.registerTempTable("wiki")
wiki.limit(2).toPandas()

,id,text,title,url
0,1,April\n\nApril is the fourth month of the year...,April,https://simple.wikipedia.org/wiki?curid=1
1,2,August\n\nAugust (Aug.) is the eighth month of...,August,https://simple.wikipedia.org/wiki?curid=2


In [41]:
categories = se.read.json("hdfs:///wiki/categories.jsonl")
categories.registerTempTable("categories")
categories.limit(2).toPandas()

,category,page_id
0,Months,1
1,Months,2


In [54]:
train_data = se.sql("""
select 
    wiki.text,
    cast(categories.category == 'American_movie_actors' as int) as target
from
    wiki join categories on wiki.id == categories.page_id
where (categories.category in ('American_movie_actors', 'American_stage_actors')) and (id % 2 = 0)
order by cast(id as int)
""")
train_data.limit(5).toPandas()

,text,target
0,Alanis Morissette\n\nAlanis Nadine Morissette ...,1
1,Ronald Reagan\n\nRonald Wilson Reagan (; Febru...,1
2,Gwyneth Paltrow\n\nGwyneth Kate Paltrow (born ...,0
3,Gwyneth Paltrow\n\nGwyneth Kate Paltrow (born ...,1
4,Heather Graham\n\nHeather Joan Graham (born Ja...,1


In [58]:
test_data = se.sql("""
select 
    wiki.text,
    cast(categories.category == 'American_movie_actors' as int) as target
from
    wiki join categories on wiki.id == categories.page_id
where (categories.category in ('American_movie_actors', 'American_stage_actors')) and (id % 2 != 0)
order by cast(id as int)
""")
test_data.limit(5).toPandas()

,text,target
0,Britney Spears\n\nBritney Jean Spears (born De...,1
1,Sarah Michelle Gellar\n\nSarah Michelle Gellar...,1
2,Jennifer Garner\n\nJenny Anne Garner (born Apr...,1
3,Christina Ricci\n\nChristina Ricci (born Febru...,1
4,Angelina Jolie\n\nAngelina Jolie (; née Voight...,1


### Топ 50000 слов по всему wiki (задача WordCount)

In [12]:
import re
import string

def tokenize(text):
    text = re.sub(f'[^{re.escape(string.printable)}]', ' ', text)  # непечатные символы заменяем на пробел
    text = re.sub(f'[{re.escape(string.punctuation)}]', ' ', text)  # и пунктуацию
    words = text.lower().split()
    return words

In [13]:
import json

def mapper(line):
    text = json.loads(line)['text']
    words = tokenize(text)
    return [(word, 1) for word in set(words)]

In [14]:
# алгоритм MapReduce
word_counts = (
    sc.textFile("hdfs:///wiki/wiki.jsonl")
    .flatMap(mapper)
    .reduceByKey(lambda a, b: a + b)
    .collect())

In [15]:
word_counts[:5]

[('out', 10522),
 ('start', 2844),
 ('where', 12648),
 ('starts', 1369),
 ('exactly', 701)]

In [16]:
top_word_counts = sorted(word_counts, key=lambda x: -x[1])[:50000]

In [17]:
top_word_counts[:5]

[('the', 143585),
 ('in', 134975),
 ('a', 134315),
 ('of', 128899),
 ('is', 125063)]

### Векторизуем тексты (модель "мешка слов") и заливаем в кэш

In [59]:
word_to_index = {word: index for index, (word, count) in enumerate(word_counts)}
word_to_index_broadcast = sc.broadcast(word_to_index)

In [60]:
def mapper(row):
    words = tokenize(row.text)
    indices = []
    values = []
    for word, count in Counter(words).items():
        if word in word_to_index_broadcast.value:
            index = word_to_index_broadcast.value[word]
            indices.append(index)
            tf = count / float(len(words))
            values.append(tf)
    return np.array(indices), np.array(values), row.target

In [61]:
train = train_data.rdd.map(mapper)
test = test_data.rdd.map(mapper)

train.cache()
test.cache()     #unpersist()

PythonRDD[1538] at RDD at PythonRDD.scala:53

In [62]:
# количество примеров
len_test = test.count()
len_train = train.count()

### Обучаем логистическую регрессию

In [69]:
def sigmoid(x):
    if x >= 0:
        return 1. / (1. + np.exp(-x))
    else:
        return np.exp(x) / (1. + np.exp(x))

In [70]:
def compute_gradient(weights_broadcast, loss, examples):
    # здесь накапливаем вклад в градиент
    gradient = np.zeros(len(weights_broadcast.value))
    
    for example in examples:
        indices, values, target = example

        # делаем предсказание с текущими весами
        p = sigmoid(values.dot(weights_broadcast.value[indices]))

        # добавляем в накопитель градиента
        gradient[indices] += values * (p - target)

        # считаем потери
        p = np.clip(p, 1e-15, 1-1e-15)
        loss.add(-(target * np.log(p) + (1 - target) * np.log(1 - p)))
    
    yield gradient

In [71]:
from functools import partial
import numpy as np

# случайные веса

np.random.seed(0); weights = np.random.random(len(word_to_index))

# эпохи градиентного спуска
for i in range(500):
    weights_broadcast = sc.broadcast(weights)
    loss = sc.accumulator(0.0)
    
    # считаем градиент
    gradient = (
        train
        .coalesce(2)  # склеиваем 200 кэшированных партиций в 2
        .mapPartitions(partial(compute_gradient, weights_broadcast, loss))
        .reduce(lambda a, b: a + b)
    )

    # обновляем веса
    weights -= 0.3 * gradient
    
    weights_broadcast.destroy()
    if i % 50 == 0:
        print("epoch:", i, "loss:", loss.value / len_train)

epoch: 0 loss: 0.61409150473
epoch: 50 loss: 0.553655212024
epoch: 100 loss: 0.538167106094
epoch: 150 loss: 0.527261777515
epoch: 200 loss: 0.518641929089
epoch: 250 loss: 0.511448695228
epoch: 300 loss: 0.505244909752
epoch: 350 loss: 0.499775344419
epoch: 400 loss: 0.494877553433
epoch: 450 loss: 0.490441197741


In [72]:
def predict(weights, examples):
    p = []
    targ = []
    for example in examples:
        indices, values, target = example

        # делаем предсказание с текущими весами
        p.append(sigmoid(values.dot(weights[indices])))
        targ.append(target)
    yield p, targ

In [73]:
predictions = (
    test
    .coalesce(2)  # склеиваем 200 кэшированных партиций в 2
    .mapPartitions(partial(predict, weights))
) .collect()


In [74]:
y_pred, y_true = np.concatenate(predictions, axis=1)

In [75]:
answer1 = roc_auc_score(y_true=y_true, y_score=y_pred)
answer1

0.68784622476892943

# Решение через хэширование

In [78]:
def mapper1(row):
    words = tokenize(row.text)
    indices = []
    values = []
    for word, count in Counter(words).items():
        if word in word_to_index:
            index = word_to_index[word]
            hash_ = murmurhash3_32(index) % 2**20
            indices.append(hash_)
            tf = count / float(len(words))
            values.append(tf)
    return np.array(indices), np.array(values), row.target

In [79]:
train1 = train_data.rdd.map(mapper1)
test1 = test_data.rdd.map(mapper1)

train1.cache()
test1.cache()     #unpersist()

PythonRDD[2550] at RDD at PythonRDD.scala:53

In [80]:
len_test1 = test1.count()
len_train1 = train1.count()

In [81]:
from functools import partial
import numpy as np

# случайные веса
np.random.seed(0)
weights = np.random.random(2**20)

# эпохи градиентного спуска
for i in range(500):
    weights_broadcast = sc.broadcast(weights)
    loss = sc.accumulator(0.0)
    
    
    # считаем градиент
    gradient = (
        train1
        .coalesce(2)  # склеиваем 200 кэшированных партиций в 2
        .mapPartitions(partial(compute_gradient, weights_broadcast, loss))
        .reduce(lambda a, b: a + b)
    )

    # обновляем веса
    weights -= 0.3 * gradient
    
    weights_broadcast.destroy()
    if i % 50 == 0:
        print("epoch:", i, "loss:", loss.value / len_train)

epoch: 0 loss: 0.617751103075
epoch: 50 loss: 0.553437494655
epoch: 100 loss: 0.538050174999
epoch: 150 loss: 0.527202030713
epoch: 200 loss: 0.518620375258
epoch: 250 loss: 0.511455428773
epoch: 300 loss: 0.505274070658
epoch: 350 loss: 0.499823051273
epoch: 400 loss: 0.494940977543
epoch: 450 loss: 0.490518128375


In [82]:
predictions1 = (
    test1
    .coalesce(2)  # склеиваем 200 кэшированных партиций в 2
    .mapPartitions(partial(predict, weights))
) .collect()


In [83]:
y_pred, y_true = np.concatenate(predictions, axis=1)

In [84]:
answer2 = roc_auc_score(y_true=y_true, y_score=y_pred)
answer2

0.68784622476892943

In [85]:
answer1 == answer2

True

In [76]:
%%file result.json
{
    "q1": 0.68784622476892943,
    "q2": 0.68784622476892943
}

Overwriting result.json


In [77]:
!cat result.json

{
    "q1": 0.68784622476892943,
    "q2": 0.68784622476892943
}

In [86]:
# останавливаем Spark (и YARN приложение)
# sc.stop()